# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
from torch_geometric.datasets import Planetoid
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(5):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_cora_graph(filename_adj, nodes_number):
    datasets = Planetoid('./datasets', filename_adj)
    edges = datasets[0].edge_index
    
    #print("edges number ： ", edges.shape[1])
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[1]):
        graph_np[edges[0,i], edges[1,i]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
filename = "Citeseer"    # The node index starts at 0, sep=None
nodes_number = 3327

In [6]:
embedding_dim = 24
n_clusters = 576
alpha = 1
beta = 4.5
DEC = 5.0
LR = 0.4

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    
    train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)
    
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(400):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.5778038758569592
AUC SCORE:  0.552670052323792
AP：  0.5990718851930069
AUC SCORE:  0.567265889504463
######################### Module cycle ： 1 ##########################
AP：  0.6140953719968737
AUC SCORE:  0.5745373576485073
AP：  0.620590415551835
AUC SCORE:  0.5788656124961528
######################### Module cycle ： 2 ##########################
AP：  0.6468555402484678
AUC SCORE:  0.5927016004924592
AP：  0.6572093494804608
AUC SCORE:  0.5975540550938752
######################### Module cycle ： 3 ##########################
AP：  0.6662765107237406
AUC SCORE:  0.6015793321021853
AP：  0.6686939122384317
AUC SCORE:  0.6029788011695907
######################### Module cycle ： 4 ##########################
AP：  0.6754749138605314
AUC SCORE:  0.6063404124345952
AP：  0.6774615028762636
AUC SCORE:  0.6071820175438597
######################### Module cycle ： 5 ##########################
AP：  0.6841787822196967
AUC SCORE

AP：  0.8062831110318789
AUC SCORE:  0.7184710680209294
######################### Module cycle ： 46 ##########################
AP：  0.8068311390428051
AUC SCORE:  0.7193463373345644
AP：  0.8071452075218147
AUC SCORE:  0.7199907663896584
######################### Module cycle ： 47 ##########################
AP：  0.8076775291016628
AUC SCORE:  0.7207890889504462
AP：  0.8080538995298496
AUC SCORE:  0.7213469529085872
######################### Module cycle ： 48 ##########################
AP：  0.8087100867538166
AUC SCORE:  0.7223616882117575
AP：  0.8092383573657327
AUC SCORE:  0.7233042859341337
######################### Module cycle ： 49 ##########################
AP：  0.8098338143497966
AUC SCORE:  0.7243430670975686
AP：  0.8102873066255527
AUC SCORE:  0.7251990997229917
######################### Module cycle ： 50 ##########################
AP：  0.810805666032758
AUC SCORE:  0.7261368882733149
AP：  0.8113436369186957
AUC SCORE:  0.727074676823638
######################### Module cycle ： 5

AP：  0.8270859816556574
AUC SCORE:  0.7629847645429362
AP：  0.8272344581051714
AUC SCORE:  0.7631963681132656
######################### Module cycle ： 92 ##########################
AP：  0.8274621943853386
AUC SCORE:  0.7636195752539243
AP：  0.8275709390082708
AUC SCORE:  0.76387446137273
######################### Module cycle ： 93 ##########################
AP：  0.8276685372063244
AUC SCORE:  0.7640235457063711
AP：  0.8277099463382613
AUC SCORE:  0.7640524007386889
######################### Module cycle ： 94 ##########################
AP：  0.8279378402523644
AUC SCORE:  0.7644515620190828
AP：  0.8280756500696145
AUC SCORE:  0.7646535472453062
######################### Module cycle ： 95 ##########################
AP：  0.8281196790858034
AUC SCORE:  0.764764158202524
AP：  0.8281517309608303
AUC SCORE:  0.7648122499230533
######################### Module cycle ： 96 ##########################
AP：  0.8284070915453502
AUC SCORE:  0.7652450754078177
AP：  0.8284925496744697
AUC SCORE:  0.76543

AP：  0.8367259166166683
AUC SCORE:  0.7807017543859649
######################### Module cycle ： 137 ##########################
AP：  0.836534680973607
AUC SCORE:  0.7805334333641121
AP：  0.8358309631292818
AUC SCORE:  0.7797639658356417
######################### Module cycle ： 138 ##########################
AP：  0.8364163825202164
AUC SCORE:  0.7805959526008003
AP：  0.8369721065928748
AUC SCORE:  0.7812163357956294
######################### Module cycle ： 139 ##########################
AP：  0.8367184287932218
AUC SCORE:  0.7810480147737766
AP：  0.8358724847706989
AUC SCORE:  0.7800188519544475
######################### Module cycle ： 140 ##########################
AP：  0.8363611728074821
AUC SCORE:  0.7807113727300709
AP：  0.8371993055607938
AUC SCORE:  0.7816972530009234
######################### Module cycle ： 141 ##########################
AP：  0.8370552840088624
AUC SCORE:  0.7816972530009234
AP：  0.8360744935089646
AUC SCORE:  0.780456486611265
######################### Module cycl

AP：  0.843005900614501
AUC SCORE:  0.7925371268082486
AP：  0.8436640200759069
AUC SCORE:  0.7934797245306248
######################### Module cycle ： 183 ##########################
AP：  0.8435981886576785
AUC SCORE:  0.7934989612188366
AP：  0.8430278291034007
AUC SCORE:  0.7927872037550014
######################### Module cycle ： 184 ##########################
AP：  0.8433468609414052
AUC SCORE:  0.7931238457987073
AP：  0.8439391735928103
AUC SCORE:  0.794018351800554
######################### Module cycle ： 185 ##########################
AP：  0.8439100946280516
AUC SCORE:  0.7940423976608187
AP：  0.8433054486712128
AUC SCORE:  0.7933594952293014
######################### Module cycle ： 186 ##########################
AP：  0.8435959508306416
AUC SCORE:  0.7936913281009541
AP：  0.844196184247185
AUC SCORE:  0.7945425515543243
######################### Module cycle ： 187 ##########################
AP：  0.8441575410912766
AUC SCORE:  0.7945858341028009
AP：  0.843558505741568
AUC SCORE:  0.7

...

AP：  0.8452118002709793
AUC SCORE:  0.7918446060326255
AP：  0.8452402831635277
AUC SCORE:  0.7919119344413665
######################### Module cycle ： 183 ##########################
AP：  0.8454068799734668
AUC SCORE:  0.7921235380116959
AP：  0.8455213193010707
AUC SCORE:  0.792272622345337
######################### Module cycle ： 184 ##########################
AP：  0.8456654045420492
AUC SCORE:  0.79252269929209
AP：  0.8457785113930935
AUC SCORE:  0.7927246845183133
######################### Module cycle ： 185 ##########################
AP：  0.8459369102189245
AUC SCORE:  0.7929410972606956
AP：  0.8460585563610845
AUC SCORE:  0.793143082486919
######################### Module cycle ： 186 ##########################
AP：  0.8461920254495279
AUC SCORE:  0.7933354493690368
AP：  0.846369344089015
AUC SCORE:  0.7936720914127423
######################### Module cycle ： 187 ##########################
AP：  0.846568145932313
AUC SCORE:  0.79394621421976
AP：  0.8467083304973612
AUC SCORE:  0.79415

AP：  0.8563509444707855
AUC SCORE:  0.8073109033548784
######################### Module cycle ： 228 ##########################
AP：  0.8564106916150447
AUC SCORE:  0.8074070867959373
AP：  0.8565156536048391
AUC SCORE:  0.8075898353339489
######################### Module cycle ： 229 ##########################
AP：  0.8566813253533537
AUC SCORE:  0.8078399122807018
AP：  0.8568228074081907
AUC SCORE:  0.8080274699907664
######################### Module cycle ： 230 ##########################
AP：  0.8569949126986344
AUC SCORE:  0.8082583102493075
AP：  0.8571181505911984
AUC SCORE:  0.8085035780240073
######################### Module cycle ： 231 ##########################
AP：  0.8572845273538061
AUC SCORE:  0.8087488457987073
AP：  0.8573671850409598
AUC SCORE:  0.8088402200677131
######################### Module cycle ： 232 ##########################
AP：  0.8574801803377945
AUC SCORE:  0.8089652585410897
AP：  0.857569591986366
AUC SCORE:  0.809099915358572
######################### Module cycl

AP：  0.8631219539966561
AUC SCORE:  0.8168090181594336
AP：  0.8631908085834954
AUC SCORE:  0.8169003924284395
######################### Module cycle ： 274 ##########################
AP：  0.8632631414123606
AUC SCORE:  0.8169965758694983
AP：  0.8633000992466745
AUC SCORE:  0.8170687134502924
######################### Module cycle ： 275 ##########################
AP：  0.8633926772767325
AUC SCORE:  0.8171793244075101
AP：  0.8634585318510037
AUC SCORE:  0.8172803170206218
######################### Module cycle ： 276 ##########################
AP：  0.8635914213984597
AUC SCORE:  0.8174534472145276
AP：  0.8636292808075037
AUC SCORE:  0.817501538935057
######################### Module cycle ： 277 ##########################
AP：  0.8636791289031065
AUC SCORE:  0.8175640581717452
AP：  0.8637355923112662
AUC SCORE:  0.8176650507848569
######################### Module cycle ： 278 ##########################
AP：  0.863803344603975
AUC SCORE:  0.817732379193598
AP：  0.8638359121981175
AUC SCORE:  0.

AP：  0.8671298776054631
AUC SCORE:  0.8220510156971376
######################### Module cycle ： 319 ##########################
AP：  0.8671689658292485
AUC SCORE:  0.8221039165897199
AP：  0.8671444072177715
AUC SCORE:  0.8220798707294552
######################### Module cycle ： 320 ##########################
AP：  0.8671770642594079
AUC SCORE:  0.8221039165897199
AP：  0.8672390660012927
AUC SCORE:  0.8221904816866729
######################### Module cycle ： 321 ##########################
AP：  0.867262324721502
AUC SCORE:  0.8222289550630963
AP：  0.8672667925823772
AUC SCORE:  0.8222530009233611
######################### Module cycle ： 322 ##########################
AP：  0.8672865795233549
AUC SCORE:  0.8222818559556787
AP：  0.8672850035631904
AUC SCORE:  0.8222674284395198
######################### Module cycle ： 323 ##########################
AP：  0.8673247989721312
AUC SCORE:  0.822329947676208
AP：  0.8673144094512706
AUC SCORE:  0.8223010926438905
######################### Module cycl

AP：  0.8693006189062168
AUC SCORE:  0.824638350261619
AP：  0.8693353706357285
AUC SCORE:  0.8247008694983071
######################### Module cycle ： 365 ##########################
AP：  0.8693566611936724
AUC SCORE:  0.8247345337026777
AP：  0.8693960809719762
AUC SCORE:  0.8247970529393659
######################### Module cycle ： 366 ##########################
AP：  0.8694199786885851
AUC SCORE:  0.8248451446598952
AP：  0.869431802189161
AUC SCORE:  0.8248788088642659
######################### Module cycle ： 367 ##########################
AP：  0.8694564032412164
AUC SCORE:  0.8248884272083717
AP：  0.8694817165887277
AUC SCORE:  0.8249365189289011
######################### Module cycle ： 368 ##########################
AP：  0.8695311919361877
AUC SCORE:  0.8250038473376424
AP：  0.8695689229184209
AUC SCORE:  0.8250615574022777
######################### Module cycle ： 369 ##########################
AP：  0.8695972775932056
AUC SCORE:  0.8250807940904894
AP：  0.8696491269661506
AUC SCORE:  0

In [8]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.8730620726704424
AP FIRST AP STD:  0.00868178899060592
AP FIRST AUC MEAN:  0.8342667936288087
AP FIRST AUC STD:  0.01113709066016623
AUC FIRST AP MEAN:  0.8730021136797383
AUC FIRST AP STD:  0.008673541691001693
AUC FIRST AUC MEAN:  0.8343677862419205
AUC FIRST AUC STD:  0.011149513057078234
